In [2]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_S_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from Algorithmes.music import estimate_angles
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
from torchsummary import summary
from numpy.linalg import eigh
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
import math
import random
from sklearn.metrics import mean_squared_error
from Algorithmes.music import generate_steering_vector
from Plots.draw_plot import plot_single_music

In [3]:
nbSources = 3 # Nombre de sources  dans l'article ils fixent le nombre de sources à 2
nbSensors = 9 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels


#Pour l'échantillon d'entraînements ils utilisent une observation de chaque combinaison d'angles d'arrivée possible pour chaque SNR level, dans le cas d'une seule source, 2G+1 observations
#SNR ratio dans le papier est seulement low, de -20 à 0 DB avec un pas de 5
#Petite note : si on fait un modèle pour chaque SNR level, les performances augmentent légérements, mais cela ne vaut pas l'effort effectué
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
T = 100
correlation_List = [0] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.
SNR_TRAIN = 30
Q = 8
N = 2**12 
theta_start = -45
theta_final = 45
phi_max = 60
rho = 1
var_ratio = [2] 
correlation_List = [0.4]

In [4]:
import numpy as np

def calculate_music_spectrum(R_hat, nbSensors, nbSources, theta_range):
    eigenvalues, eigenvectors = np.linalg.eigh(R_hat)
    noise_subspace = eigenvectors[:, :nbSensors - nbSources]
    music_spectrum = np.zeros(len(theta_range))

    for i, theta in enumerate(theta_range):
        steering_vector = generate_steering_vector(nbSensors, theta)
        music_spectrum[i] = 1 / np.abs(np.conj(steering_vector).T @ noise_subspace @ noise_subspace.conj().T @ steering_vector)

    return music_spectrum

perturbation_parameter_sd = 0

In [5]:
import itertools

def generate_combinations(phi_max, rho, nb_sources):
    """Cette fonction sert à générer toutes les combinaisons d'angles possibles dans la plage de résolution"""
    # Générer une plage de valeurs possibles pour les signaux
    values = list(range(-int(phi_max / rho), int(phi_max / rho) + 1))
    
    # Générer toutes les combinaisons possibles de signaux
    all_combinations = list(itertools.product(values, repeat=nb_sources))
    
    # Supprimer les combinaisons où l'ordre ne compte pas et où deux signaux ont la même valeur
    unique_combinations = {tuple(sorted(combination)) for combination in all_combinations if len(set(combination)) == nb_sources}
    
    return list(unique_combinations)


# Pour var_ratio et correlation, faites attention à bien avoir une liste de liste de réels s'ils sont en parameter_to_compare, et une liste de réels sinon.
nbTimePoints = 100
var_ratio = [1]
correlation_List = [0]
snr = [-10, 0, 10, 20]
perturbation_parameter_sd = 0
nb_iterations = 100 # Nombre d'itérations de Monte Carlo pour chaque algorithme
nbSources = 2
nbSensors = 9
phi_max = 60
Q = 8
N = 2**12 
phi_max = 10
rho = 1


def generate_deepmusic_partitioned_data(nbSensors, nbSources, T, SNR_TRAIN, Q, N, phi_max, rho, correlation_List, var_ratio):
    angles_combination = generate_combinations(phi_max, rho, nbSources)
    training_data_sets = [[] for _ in range(Q)]
    L = N // Q  # Assuming N is divisible by Q

    i = 0
    for signal_noise_ratio in SNR_TRAIN:
        for angles in angles_combination:
            thetaList = list(angles)

            X = generate_X_matrix(nbSources, nbSensors, nbTimePoints, thetaList, var_ratio, correlation_List, signal_noise_ratio, perturbation_parameter_sd)
            R_hat_with_phase = generate_R_hat_with_phase(X)
            R_hat = generate_R_hat(X)

            full_theta = np.linspace(- phi_max , phi_max, N)
            full_music_spectrum = calculate_music_spectrum(R_hat, nbSensors, nbSources, full_theta)

            for q in range(Q):
                # Sampling MUSIC spectrum for the q-th subregion
                start_index = q * L
                end_index = start_index + L
                pq = full_music_spectrum[start_index:end_index]
                
                input_data = R_hat_with_phase
                output_data = pq
                training_data_sets[q].append((input_data, output_data))
            i += 1

    return training_data_sets

full_theta = np.linspace(- phi_max , phi_max, N)
# Example usage
SNR_TRAIN = [-20,-15,-10, -5, 0] # Different SNR levels for training
training_datasets = generate_deepmusic_partitioned_data(nbSensors, nbSources, T, SNR_TRAIN, Q, N, phi_max, rho, correlation_List, var_ratio)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class DeepMusicDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_data, output_data = self.data[idx]
        return torch.tensor(input_data, dtype=torch.float32), torch.tensor(output_data, dtype=torch.float32)

def create_loaders_for_subregion(data, batch_size=128, validation_split=0.2, test_split=0.2):
    # Séparation des données en ensembles d'entraînement, de validation et de test
    train_data, test_data = train_test_split(data, test_size=test_split, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=validation_split, random_state=42)

    # Création des instances de DeepMusicDataset
    train_dataset = DeepMusicDataset(train_data)
    val_dataset = DeepMusicDataset(val_data)
    test_dataset = DeepMusicDataset(test_data)

    # Création des DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Création des DataLoader pour chaque sous-région
loaders_per_subregion = [create_loaders_for_subregion(training_datasets[q]) for q in range(Q)]

NameError: name 'Q' is not defined

In [ ]:

class DeepMusicModel(nn.Module):
    def __init__(self):
        super(DeepMusicModel, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=256)
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=256)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        
        # Assuming the spatial dimensions (height and width) are reduced to 1x1 after the convolutions
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=20736, out_features=512)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Softmax layer 
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply the first convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Apply the second convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Apply the third convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Apply the fourth convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        
        # Apply the first fully connected layer
        x = self.fc1(x)
        
        # Apply the dropout layer
        x = self.dropout(x)

        #Apply the softmax layer
        x = self.softmax(x)
        
        return x

model = DeepMusicModel()
model

DeepMusicModel(
  (conv1): Conv2d(3, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=20736, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)

In [ ]:
#Définition du device

device = (
    "gpu"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [ ]:
import torch
import time
import numpy as np
from scipy.signal import find_peaks
import torch.nn.functional as F

def calculate_accuracy(predicted_spectrum, target_spectrum, angles_range, nbSources, peak_tolerance):
    """
    Calculate accuracy for a single predicted and target spectrum.
    
    :param predicted_spectrum: Predicted MUSIC spectrum (1D NumPy array).
    :param target_spectrum: True MUSIC spectrum (1D NumPy array).
    :param angles_range: The range of angles over which the MUSIC spectrum is calculated.
    :param nbSources: The number of sources (peaks) to consider.
    :param peak_tolerance: The tolerance for considering a predicted peak to match a target peak.
    :return: Accuracy as a float.
    """
    # Estimate angles from the predicted and target spectra
    estimated_angles = estimate_angles(nbSources, predicted_spectrum, angles_range)
    true_angles = estimate_angles(nbSources, target_spectrum, angles_range)

    # Compare the estimated angles with the true angles
    correct_predictions = sum(1 for est_angle in estimated_angles if np.any(np.abs(true_angles - est_angle) <= peak_tolerance))
    accuracy = correct_predictions / nbSources if nbSources > 0 else 0
    return accuracy



def train_and_evaluate_spectrum(loaders_per_subregion, epochs, nbSources, angles_range, peak_tolerance=5):
    models = []
    train_loss_list = []
    val_loss_list = []
    val_mse_list = []

    # Training configuration
    initial_lr = 0.01
    lr_decay_factor = 0.5
    lr_decay_epoch = 10
    early_stopping_patience = 3

    start_time = time.time()  # Start time for training

    for train_loader, val_loader, test_loader in loaders_per_subregion:
        model = DeepMusicModel()
        optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=0.9)
        criterion = torch.nn.MSELoss()

        best_val_mse = float('inf')
        epochs_no_improve = 0

        for epoch in range(epochs):
            # Adjust learning rate
            if epoch % lr_decay_epoch == 0 and epoch > 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= lr_decay_factor

            # Training loop
            model.train()
            train_loss = 0
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets).item()
                optimizer.step()
                train_loss += loss
                
            avg_train_loss = train_loss / len(train_loader)
            train_loss_list.append(avg_train_loss)

            # Validation loop
            model.eval()
            val_loss, val_mse = 0, 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
                    val_mse += F.mse_loss(outputs, targets).item()

            avg_val_loss = val_loss / len(val_loader)
            avg_val_mse = val_mse / len(val_loader)
            val_loss_list.append(avg_val_loss)
            val_mse_list.append(avg_val_mse)

            # Early stopping
            if avg_val_mse < best_val_mse:
                best_val_mse = avg_val_mse
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve == early_stopping_patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        models.append(model)

    # Calculate training time
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time} seconds")

    # Test set evaluation
    concatenated_outputs = []
    concatenated_targets = []

    for model, test_loader in zip(models, [x[2] for x in loaders_per_subregion]):
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)

                # Concatenate outputs and targets
                concatenated_outputs.append(outputs)
                concatenated_targets.append(targets)

    # Calculate the average MSE per epoch on the validation set
    avg_mse_per_epoch_val = []
    for epoch in range(epochs):
        epoch_mse = []
        for i, model in enumerate(models):
            # Determine the actual number of epochs the model was trained for
            actual_epochs = len(train_loss_list[i * epochs:(i + 1) * epochs])
            if epoch < actual_epochs:
                epoch_mse.append(train_loss_list[i * epochs + epoch])
        avg_mse_per_epoch_val.append(np.mean(epoch_mse))

    print("Average MSE per epoch on validation set:", avg_mse_per_epoch_val)

    # Calculate the average MSE per epoch on the training set
    avg_mse_per_epoch_train = []
    for epoch in range(epochs):
        epoch_mse = []
        for i, model in enumerate(models):
            # Determine the actual number of epochs the model was trained for
            actual_epochs = len(val_mse_list[i * epochs:(i + 1) * epochs])
            if epoch < actual_epochs:
                epoch_mse.append(val_mse_list[i * epochs + epoch])
        avg_mse_per_epoch_train.append(np.mean(epoch_mse))

    print("Average MSE per epoch on training set:", avg_mse_per_epoch_train)

    # Concatenate outputs and targets over all batches
    concatenated_outputs = torch.cat(concatenated_outputs, dim=0)
    concatenated_targets = torch.cat(concatenated_targets, dim=0)

    # Compute MSE for concatenated outputs and targets
    total_mse = F.mse_loss(concatenated_outputs, concatenated_targets).item()

    print(f'Average Training Loss: {np.mean(train_loss_list):.4f}')
    print(f'Average Validation Loss: {np.mean(val_loss_list):.4f}')
    print(f'MSE on Test Set: {total_mse:.4f}')
    print(f'Training time: {training_time}')
    return models, {"MSE" : total_mse}
# Example usage
model1_results = train_and_evaluate_spectrum(loaders_per_subregion=loaders_per_subregion, epochs=1000, nbSources=nbSources, angles_range=full_theta, peak_tolerance=5)

Early stopping at epoch 16
Early stopping at epoch 4
Early stopping at epoch 14
Early stopping at epoch 4
Early stopping at epoch 8
Early stopping at epoch 4
Early stopping at epoch 7
Early stopping at epoch 4
Training time: 69.46613621711731 seconds
Average MSE per epoch on validation set: [2.295368194580078, 2.6155027747154236, 2.7866031924883523, 2.192793071269989, 2.221536854902903, 2.2817610104878745, 2.288720746835073, 2.2600502371788025, 2.344677289326986, 2.5646402637163797, 2.851960778236389, 2.3428588211536407, 2.1864139338334403, 2.592466155687968, 2.8030747373898826, 2.448508898417155, 3.5667812824249268, 4.372400363286336, 4.598643660545349, 4.40651257832845, 4.191354990005493, 3.7875811656316123, 4.033564448356628, 5.009076158205668, 4.500150680541992, 3.60952823360761, 4.510543346405029, 3.7876556317011514, 3.862421194712321, 4.391455411911011, 4.644663691520691, 4.237703005472819, 3.963491678237915, 3.538557385404905, 3.6219451626141868, 4.062512159347534, 3.80520315965

C:\Users\natha\AppData\Roaming\Python\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\natha\AppData\Roaming\Python\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
from Models.compare import *
models, _ = model1_results
save_models_hyperparams_and_metadata(models, hyperparameters_list=[{"A" : "a"} for _ in range(Q)], metadata_list=[{"A" : "a"} for _ in range(Q)], directory_name="DeepMusictest")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\natha\AppData\Roaming\Python\Python311\site-packages\torch\serialization.py", line 619, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol, _disable_byteorder_record)
  File "C:\Users\natha\AppData\Roaming\Python\Python311\site-packages\torch\serialization.py", line 853, in _save
    zip_file.write_record(name, storage.data_ptr(), num_bytes)
RuntimeError: [enforce fail at inline_container.cc:593] . PytorchStreamWriter failed writing file data/28: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\natha\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\natha\AppData\Local\Temp\ipykernel_54756\1410189850.py", line 3, in <module>
    save_models_hyperparams_and_metadata(models, hyperparameters_list=[{"A" : "

In [ ]:
models3

([DeepMusicModel(
    (conv1): Conv2d(3, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=20736, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (softmax): Softmax(dim=1)
  ),
  DeepMusicModel(
    (conv1): Conv2d(3, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0

+-